In [8]:
# urls=[
#     "https://www.cnbc.com/",
#     'http://cnn.com',
# ]

# for url in urls:
#     paper = newspaper.build(url)
#     for article in paper.category_urls():
#         print(article.text)

In [9]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
import hashlib

class RSS_Item(object):
    def __init__(self,item:Tag,publisher:str) -> None:
        self.publisher = publisher
        self.item = item
        self.link = item.find('link').text
        self.pub_date = item.find('pubDate').text
        self.tickers = None
        #Seeking Alpha gives us the tickers of the article
        if item.find('category'):
            categories = item.find_all('category')
            self.tickers = [category.text for category in categories]
            
    @property
    def hash(self)->str:
        return hashlib.sha512((self.publisher+self.link).encode("UTF-8")).hexdigest()
            
         
publisher = 'seekingalpha'   
feed = "https://seekingalpha.com/market_currents.xml"
# first make a get request to the RSS feed
response = requests.get(feed)
# collect the contents of the request
webpage = response.content
# create a BeautifulSoup object that we can then parse to extract the links and title
soup = BeautifulSoup(webpage, features='xml')
# here we find every instance of an <item> tag, collect everything inside each tag, and store them all in a list
items = soup.find_all('item')
# extract the article link within each <item> tag and store in a separate list
rss_items = []
for item in items:
    rss_items.append(RSS_Item(item,publisher))

In [10]:
print([rss_item.hash for rss_item in rss_items])
                    

['256cd48b10d576fa4bf952304e2133cfa738c729b20bb64dbf23870c4f3b977438faf93fe4525ba831ac6320b8401143a11435ab07e5dbe61fef383e04ee22f4', '9803e0055ae6bc5ea729bf4b3d8514f3900cfc304c66bdd9735255efdd0c35cbf730a40bcee3b3eb846f94e69d8979bae68696b4f0d7ec420001ddeabc1d1c5a', '44b5fa9ec7f3b19766f62af47a4dc3bdbeb4017a7d6cacf6e9d143f4e904567e6d7bd8317cdc79b9eb16fd21567e5e5c7a39cf6d919fd2ee0379cc8fb2f04db9', 'ade1b6f2ad2e621abe6c0b64b336ee998d4b4c261ea12f832b008a074443c3bbf6482c31a49f51711460511d73ec95f74d238c99ecf637c8deffc5752ea26db1', 'c26e1248a2dfc3c1458f605000873cd8afb67224334e680a872d76d8540af95ab040e9a21f8448071c149c13346d12107cb1ff5ce5a67c7eaa7f163fb1790fb9', '0d9d0f307372245414744f82be348c2a207c6850970076ae423d751ac267164ad1861558d663d05f772f102e0f784e73964533992c786bdb06c4b6d087c37c3a', '0bb7ee02fe9ba940489ed83b002df0f5847e7a9022ec9f8c5634e54fcf64a0ccdb6977792438639a493d858dccd3beba9e47cf26a72072c0c980ea2c612b5083']


In [12]:
from tqdm import tqdm 
from newspaper import Article

for rss_item in rss_items:
    article = Article(rss_item.link)
    article.download()
    article.parse()


ArticleException: Article `download()` failed with 403 Client Error: Forbidden for url: https://seekingalpha.com/warnings/3863488-warning-amkby-is-high-risk-of-cutting-dividend?utm_source=feed_news_all&utm_medium=referral on URL https://seekingalpha.com/warnings/3863488-warning-amkby-is-high-risk-of-cutting-dividend?utm_source=feed_news_all&utm_medium=referral